In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()

VECTOR_STORE_NAME = "rule_demo"

vector_store = client.vector_stores.create(
    name=VECTOR_STORE_NAME,
    # 필요하면 metadata, description 등도 추가 가능
)

print(vector_store.id)


vs_694b717ccf54819196930e17c605668f


In [ ]:
chunks = [
    {
        "id": "doc1-chunk0",
        "content": "결제 수단은 계정 설정의 결제 정보에서 변경할 수 있습니다...",
        "metadata": {"doc_id": "doc1", "section": "billing", "lang": "ko"},
    },
    # ...
]

for chunk in chunks:
    client.vector_stores.items.create(
        vector_store_id=vector_store.id,
        input=chunk["content"],
        metadata=chunk["metadata"],
        id=chunk["id"],
        # embedding은 내부에서 자동 생성되도록 맡기거나
        # 미리 embedding 모델을 지정할 수도 있음
    )

In [ ]:
results = client.vector_stores.query(
        vector_store_id=vector_store.id,
        input=query,
        top_k=k,
        # 필요하면 filter에 metadata 조건을 줄 수도 있다 (예: lang == 'ko')
    )

In [15]:
# Vector Store 생성
vector_store = client.vector_stores.create(
  name="knou_rules",
  metadata={
    "description": "한국방송통신대학교 학칙"
   },
  chunking_strategy={
    'type': 'static',
    'static': {
      'max_chunk_size_tokens': 800,
      'chunk_overlap_tokens': 400
    }
  } # 또는 auto
)

In [ ]:
vector_store_list = client.vector_stores.list()
print(vector_store_list)

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_694b742456148191be27d5686dc95cb9', created_at=1766552612, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1766552620, metadata={'description': '한국방송통신대학교 학칙'}, name='knou_rules', object='vector_store', status='completed', usage_bytes=263319, expires_after=None, expires_at=None, description=None), VectorStore(id='vs_694b739384c481918a28ed3918831bf2', created_at=1766552467, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1766552598, metadata={'description': '한국방송통신대학교 학칙'}, name='knou_rules', object='vector_store', status='completed', usage_bytes=263319, expires_after=None, expires_at=None, description=None), VectorStore(id='vs_694b717ccf54819196930e17c605668f', created_at=1766551933, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1766551933, metadata={}, name='rule_demo', object='vec

In [16]:
# 파일 업로드
file = client.files.create(
    file=open("한국방송통신대학교 학칙.pdf", "rb"),
    purpose="assistants"
)

# Vector Store에 파일 추가
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)

VectorStoreFile(id='file-KtUGmwD8RYJEiyYDtQNdPW', created_at=1766554040, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_694b79aef7d081919879ef352d9ba001', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [ ]:
query = "방송통신대에는 어떤 학과가 있나요?"

# 벡터스토어 검색 수행
search_results = client.vector_stores.search(
  vector_store_id=vector_store.id,
  query=query,
  max_num_results=5
)

for result in search_results.data:
  print(result.score, result.content)

print(search_results.data[0].content[0].text)



0.7445442274378524 [Content(text='10. 1. 규칙 제1076호)\r\n제1조(시행일) 이 학칙은 공포한 날부터 시행한다. 다만, [별표2] 학과별 1학년 입학정원과 3학년 편\r\n입학정원, [별표3] 대학원 설치 학과 및 입학정원, [별표4] 경영대학원 설치 전공 및 입학정원, [별\r\n표5] 프라임칼리지 설치학부 및 입학정원은 2020학년도부터 적용한다.\r\n제2조(학과명칭 변경에 따른 경과조치) ① 이 학칙 시행 당시 불어불문학과 및 환경보건학과 재적생\r\n은 프랑스언어문화학과 및 보건환경학과의 재적생으로 본다.\r\n② 이 학칙 시행 전에 불어불문학과 및 환경보건학과에서 제적 또는 자퇴한 사람이 재입학하려는- 63 - ◀한국방송통신대학교 학칙▶\r\n때에는 프랑스언어문화학과 및 보건환경학과에서 제적한 것으로 본다.\r\n제3조(다른 규정과의 관계) 이 학칙 시행 당시 본교의 다른 규정에서 이 학칙으로 변경되는 종전의 학\r\n과명, 전공명 또는 학위명을 인용한 경우에는 이 학칙의 해당 학과명, 전공명 또는 학위명을 인용\r\n한 것으로 본다.\r\n부 칙(2020. 5. 21. 규칙 제1113호)\r\n제1조(시행일) 이 학칙은 2020학년도 1학기부터 시행한다.\r\n제2조(학과명칭 변경에 따른 경과조치) ① 이 학칙 시행 당시 생활과학과 재적생은 생활과학부 재적\r\n생으로 본다.\r\n② 이 학칙 시행 전에 생활과학과에서 제적 또는 자퇴한 사람이 재입학하려는 때에는 생활과학부\r\n에서 제적한 것으로 본다.\r\n제3조(다른 규정과의 관계) 이 학칙 시행 당시 본교의 다른 규정에서 이 학칙으로 변경되는 종전의\r\n학과명, 전공명 또는 학위명을 인용한 경우에는 이 학칙의 해당 학부명, 전공명 또는 학위명을 인\r\n용한 것으로 본다.', type='text')]
0.7321346847491434 [Content(text='부 칙(2020. 5. 21. 규칙 제1113호)\r\n제1조(시행일) 이 학칙은 2020학년

In [ ]:
question = "방송통신대에는 어떤 학과가 있나요?"

response = client.responses.create(
    model="gpt-4.1-mini",
    input=question,
    instructions="당신은 학사정보 상담사입니다. 제공된 문서를 기반으로 정확한 답변을 제공해주세요.",
    temperature=0.2,
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store.id],
            "max_num_results": 10
        }
    ],
    # include=["file_search_call.results"]

)

print(response.output_text)
print(response.output[0].results[0].text)

한국방송통신대학교에는 여러 단과대학과 학과(부)가 설치되어 있습니다. 2023년 학칙 기준으로 주요 학과는 다음과 같습니다.

1. 인문과학대학
- 국어국문학과
- 영어영문학과
- 중국어중문학과
- 프랑스언어문화학과
- 일본학과

2. 사회과학대학
- 법학과
- 행정학과
- 경제학과
- 경영학과
- 무역학과
- 미디어영상학과
- 관광학과
- 사회복지학과

3. 대학원 학과
- 문예창작콘텐츠학과
- 실용영어학과
- 실용중국어학과
- 아프리카·불어권언어문화학과
- 일본언어문화학과
- 법학과
- 행정학과
- 영상문화콘텐츠학과
- 사회복지학과
- 농업생명과학과
- 생활과학과
- 정보과학과
- 에듀테크학과
- 통계·데이터과학과
- 환경보건시스템학과
- 간호학과
- 평생교육학과
- 청소년교육학과
- 유아교육학과

4. 프라임칼리지 설치학부·전공
- 융합경영학부: 회계금융전공, 마케팅·에널리틱스전공
- 첨단공학부: 메카트로닉스전공, 산업공학전공, AI전공

이 외에도 생활과학부(가정학과에서 개편), 계약학과(부) 등이 있으며, 지역대학별로 학습센터 및 학습관이 운영되고 있습니다.

자세한 학과별 입학정원과 조직은 학칙 별표에 명시되어 있습니다【1:1~2†한국방송통신대학교 학칙.pdf】【6:6~7†한국방송통신대학교 학칙.pdf】.


TypeError: 'NoneType' object is not subscriptable